In [89]:
from bs4 import BeautifulSoup
import requests

In [102]:
url = 'https://github.com/chungyiweng/chungyiweng.github.io/commits/main/'
page = requests.get(url)
print(page)

<Response [200]>


In [91]:
OWNER = "chungyiweng" #username
REPO = "chungyiweng.github.io"  #repo
BRANCH = "main" #compare to 
API_BASE_URL = f"https://api.github.com/repos/{OWNER}/{REPO}"

In [92]:
url = f"{API_BASE_URL}/commits?sha={BRANCH}"
response = requests.get(url)
commits = response.json()
commits_len= len(commits)
#commits[0] is the latest commit.

In [93]:
import json

def extract_old_new_from_patch(patch_text):
    old_version = []
    new_version = []
    
    for line in patch_text.split("\n"):
        if line.startswith("---") or line.startswith("+++"):  # Ignore file headers
            continue
        elif line.startswith("-") and not line.startswith("--"):
            old_version.append(line[1:].strip())  
        elif line.startswith("+") and not line.startswith("++"):
            new_version.append(line[1:].strip())  
    
    return old_version, new_version

In [95]:
#function that compares two commits and returns dictionary
def compare_commit(commit1, commit2):
    url1 = f"{API_BASE_URL}/compare/{commit2}...{commit1}"
    response1 = requests.get(url1)
    diff_data = response1.json()    
    return diff_data


In [107]:
import pandas as pd
#compare every two adjacent commits:
df_list = []
for i in range (0,commits_len-1):
    #compare commits[i] and commits[i+1]
    commit1 = commits[i]['sha']
    commit2 = commits[i+1]['sha']
    diff_data = compare_commit(commit1, commit2)
    column_names = ['url', 'commit message', 'file name', 'file status', 'page before commit', 'section modified', 'page after commit', 'new section']
    url = 'https://github.com/chungyiweng/chungyiweng.github.io/commits/main/'
    message = diff_data['commits'][0]['commit']['message']
    if "files" in diff_data:
        files = diff_data.get("files", [])
        for file in files:
            file_name = file['filename']
            url1 = f"https://raw.githubusercontent.com/{OWNER}/{REPO}/{commit2}/{file_name}"
            url2 = f"https://raw.githubusercontent.com/{OWNER}/{REPO}/{commit1}/{file_name}"
            status = file['status']
            if "patch" in file:
                    patch1 = file["patch"]
                    diff_tuple = extract_old_new_from_patch(patch1)
                    old = diff_tuple[0]
                    new = diff_tuple[1]
                    print(requests.get(url1))
                    if requests.get(url1).status_code != 200:
                        html_before = "page does not exist"
                    else:
                        html_before = requests.get(url1).text
                    html_after = requests.get(url2).text
                    soup_before = BeautifulSoup(html_before, "html.parser").prettify()
                    soup_after =  BeautifulSoup(html_after, "html.parser").prettify()
                    df_list.append([url, message, file_name, status, soup_before, old, soup_after, new])
            else:
                    patch1= "no changes"
                    df_list.append([url, message, file_name, status, "", "", "", ""])
            
            
        
                 
    #each commmit could have more than one file modified
    #extract data from diff_data and upload to pd df
df = pd.DataFrame(df_list, columns =column_names)
df.head(10)
df.to_csv("data.csv", index=False)
#handle cases where patch is empty

<Response [200]>
<Response [404]>
<Response [200]>
